In [14]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [15]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=2, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [16]:
def calcBc():
    r = 1/Integrate(1,mesh)*Integrate(x,mesh)
    return r
calcBc()

-5.080028269950915e-17

In [17]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)

In [19]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

dJOmega  = LinearForm(VEC)

# volume stuff
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)
vol = Parameter(1)
vol.Set(Integrate(surf_t,mesh))
vol_inv = Parameter(1)
vol_inv.Set(1/Integrate(surf_t,mesh))
# bc prep
bc_tx = CoefficientFunction(x)
bc_0x = 1/surf_0*Integrate(bc_tx,mesh)

# bc constraints
beta0 = 1e-1
beta = Parameter(beta0)
bc_x = Parameter(1)
bc_int = Parameter(1)
bc_int.Set(Integrate(bc_tx,mesh))
bc_x.Set(calcBc())

dJOmega += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)

In [20]:
dJOmega.Assemble()
print("curvol: ", vol.Get())
print("Initial bc: ", bc_x.Get())
print(dJOmega.vec[0:5])

curvol:  12.566430839284152
Initial bc:  -5.080028269950915e-17
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [21]:
mesh.UnsetDeformation()
test = CoefficientFunction((0.8,1.2))
gfX.Set(test)
mesh.SetDeformation(gfX)
scene = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [22]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
dJOmega.Assemble()
print("curvol: ", Integrate(1,mesh))
print(bc_x.Get())
print(abs(bc_0x-bc_x.Get()))
#print(dJOmega.vec)

curvol:  12.56643083928415
0.7999999999999996
0.7999999999999996


## try to do this as shape optimization
Barycenter:
$\Large bc = \frac{1}{\int_\Omega 1 \mathrm{dx}}\cdot\int_\Omega x \mathrm{dx}$ <br>
#### above as shape derivative:
$\Large-\frac{1}{|\Omega |^2}\cdot\int div(x)\mathrm{dx}\cdot(\int x \mathrm{dx})+$<br>
$\Large\frac{1}{|\Omega |}\cdot\int div(v)x_i \mathrm{dx}\cdot\int v_i \mathrm{dx}$ <br>

In [23]:
# don't use 2 deformations, just set bc_0x to something else, let's see if we get there:
mesh.UnsetDeformation()
bc_0x = 1.5

gfset = GridFunction(VEC)

dJOmega2 = LinearForm(VEC)
# initial try in auto (wrong)
#dJOmega2 += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)
# derivative by sturm (wrong)
#dJOmega2 += ((2*beta*((1/bc_x-bc_0x))*(1/bc_x)*dx)).DiffShape(X)
# another try (wrong)
# dJOmega2 += (2*(1/(vol_inv*bc_int)-bc_0x)*(1/(vol_inv*bc_int))*dx).DiffShape(X)
# new try
dJOmega2 += 2*beta*(bc_x-bc_0x)*((1/vol**2)*div(X)*x+(1/vol)*div(X)*x*sum(gfset.vecs[0].data)[0])*dx

gfset = GridFunction(VEC)

b = BilinearForm(VEC)
#b += (InnerProduct(grad(X),grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx
b += (InnerProduct(grad(X)+grad(X).trans,grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega2.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [25]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
bc_x.Get()

-5.080028269950915e-17

In [26]:
import time
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
iter_max = 100

for i in range(0,iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    b.Assemble()
    
    vol_inv.Set(1/Integrate(surf_t,mesh))
    vol.Set(Integrate(surf_t,mesh))
    bc_x.Set((1/vol.Get())*Integrate(bc_tx,mesh)) # needed for multiple iterations (already done once before)
    bc_int.Set(Integrate(bc_tx,mesh))
    #print(1/Integrate(1,mesh)*Integrate(x,mesh))
    print(bc_x.Get())
    dJOmega2.Assemble()
    SolveDeformationEquation()
    
    mesh.UnsetDeformation()
    
    gfxnorm = Norm(gfX.vec)
    scale = 0.1 / gfxnorm
    gfset.vec.data -= scale * gfX.vec
    
    time.sleep(0.1)

-5.080028269950915e-17
0.02343659597493792
0.046866434835170265
0.07028781954400685
0.09369906261353192
0.11709848741137051
0.14048442959105104
0.16385523862162735
0.18720927939009757
0.21054493384877376
0.23386060267912728
0.2571547069437334
0.2804256896987187
0.30367201754053946
0.32689218206288767
0.35008470120193735
0.3732481204509493
0.39638101392829733
0.41948198528620056
0.4425496684507316
0.46558272818692653
0.48857986048599306
0.5115397927746146
0.5344612839491013
0.5573431242396759
0.580184134912379
0.6029831678179827
0.6257391047988947
0.6484508569662709
0.6711173638605349
0.6937375925091295
0.716310536395719
0.738835214355193
0.7613106694087018
0.7837359675526739
0.8061101965153136
0.828432464493411
0.8507018988816337
0.8729176450055965
0.8950788648691275
0.9171847359252027
0.9392344498790527
0.9612272115309424
0.9831622376651218
1.0050387559905205
1.0268560041377304
1.0486132287159704
1.0703096844328026
1.0919446332785767
1.1135173437767634
1.1350270903006523
1.15647315245

In [12]:
bc_x.Get()

1.4998614940847228